# No Show EDA

Exploratory data analysis of first datadump of no-show

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from noshow.config import CLINIC_CONFIG
from noshow.features.appointment_features import (
    add_appointments_last_days,
    add_appointments_same_day,
    add_days_since_created,
    add_days_since_last_appointment,
    add_minutes_early,
    add_time_features,
)
from noshow.features.no_show_features import prev_no_show_features
from noshow.features.patient_features import add_patient_features
from noshow.preprocessing.load_data import (
    load_appointment_csv,
    process_appointments,
    process_postal_codes,
)
from noshow.visualisation.features_plots import feature_barplot, feature_scatter

## Read and clean data

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_no_show.csv")
appointments_df = process_appointments(appointments_df, CLINIC_CONFIG)
appointments_df.shape

## Plot no-shows

In [ ]:
mutation_reason_count = (
    appointments_df[appointments_df["no_show"] == "no_show"]
    .groupby("mutationReason_display")["no_show"]
    .count()
)

fig, ax = plt.subplots(figsize=(10, 8))

bar_container = ax.bar(mutation_reason_count.index, mutation_reason_count)
ax.bar_label(bar_container)
ax.set_xticks(range(len(mutation_reason_count.index)))
ax.set_xticklabels(mutation_reason_count.index, rotation=45, ha="right")
ax.set_title("Mutation reasons for no-shows")
fig.set_layout_engine("tight")
plt.show()

In [ ]:
no_show_per_specialisation = (
    appointments_df.groupby("clinic")["no_show"].value_counts().unstack()
)

bottom = np.zeros(2)
fig, ax = plt.subplots(figsize=(10, 4))
for idx, row in no_show_per_specialisation.iterrows():
    ax.bar(["no-show", "show"], row, label=idx, bottom=bottom)
    bottom += row
ax.legend()

plt.show()

In [ ]:
no_show_percent = (
    appointments_df.groupby("no_show")["clinic"]
    .value_counts(normalize=True)
    .unstack(level="no_show")
)
fig, ax = plt.subplots(figsize=(10, 4))

bottom = np.zeros(2)
for idx, row in no_show_percent.iterrows():
    ax.bar(["no-show", "show"], row, label=idx, bottom=bottom)
    bottom += row

ax.legend()

plt.show()

In [ ]:
no_show_over_time = appointments_df.copy().reset_index()
no_show_over_time["start"] = (
    no_show_over_time["start"].dt.to_period("M").dt.to_timestamp()
)
no_show_over_time = (
    no_show_over_time.groupby(["clinic", "start"])["no_show"]
    .value_counts(normalize=True)
    .unstack(level="no_show")
)

fig, ax = plt.subplots(figsize=(12, 5))
for idx in no_show_over_time.index.unique(level="clinic"):
    ax.plot(
        no_show_over_time.loc[idx].index,
        no_show_over_time.loc[idx, "no_show"],
        label=idx,
    )
ax.legend()
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
ax.set_title("Percentage of no-shows over time")

plt.show()

In [ ]:
no_show_over_time = appointments_df.copy().reset_index()
no_show_over_time["start"] = (
    no_show_over_time["start"].dt.to_period("M").dt.to_timestamp()
)
no_show_over_time = no_show_over_time.groupby(["clinic", "start"])["no_show"].count()

fig, ax = plt.subplots(figsize=(12, 5))
for idx in no_show_over_time.index.unique(level="clinic"):
    ax.plot(
        no_show_over_time.loc[idx].index,
        no_show_over_time.loc[idx],
        label=idx,
    )
ax.legend()
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
ax.set_title("number of appointments over time")

plt.show()

In [ ]:
appointments_df["month"] = appointments_df.index.get_level_values("start").month
month_names = [
    "jan",
    "feb",
    "mrt",
    "apr",
    "mei",
    "jun",
    "jul",
    "aug",
    "sep",
    "okt",
    "nov",
    "dec",
]

no_show_month = (
    appointments_df.groupby(["month", "clinic"])["no_show"]
    .value_counts(normalize=True)
    .unstack(level="no_show")
)
fig, ax = plt.subplots(3, figsize=(10, 4))

for idx, clinic in zip(
    range(3), no_show_month.index.unique(level="clinic"), strict=False
):
    plot_data = no_show_month.loc[(slice(None), clinic), "no_show"]
    ax[idx].bar(month_names, plot_data, label=clinic)
    ax[idx].legend()

ax[0].set_title("Average percentage of no-show per month")
fig.set_layout_engine("tight")
plt.show()

## Calculate earlier no-shows

In [ ]:
appointments_features = prev_no_show_features(appointments_df)

In [ ]:
ax = feature_barplot(
    appointments_features,
    "prev_no_show",
    feature_name="previous no-shows",
)
plt.tight_layout()
plt.show()

In [ ]:
feature_barplot(
    appointments_features,
    "prev_no_show_perc",
    feature_name="previous no-show percentage",
    perc_feature=True,
    round_decimals=1,
)
plt.show()

In [ ]:
ax = feature_scatter(appointments_features, "earlier_appointments", figsize=(18, 8))
plt.tight_layout()
plt.show()

### Calculate days since created

In [ ]:
appointments_features = add_days_since_created(appointments_features)

In [ ]:
feature_scatter(appointments_features, "days_since_created")
plt.show()

### Calculate appointments on the same day

In [ ]:
feature_barplot(appointments_features, "minutesDuration")
plt.show()

In [ ]:
appointments_features = add_appointments_same_day(appointments_features)

In [ ]:
feature_barplot(appointments_features, "appointments_same_day")
plt.show()

In [ ]:
appointments_features = add_days_since_last_appointment(appointments_features)
appointments_features = add_appointments_last_days(appointments_features)

In [ ]:
ax = feature_scatter(appointments_features, "days_since_last_appointment")
ax[1].set_ylim(0, 1000)
plt.show()

In [ ]:
feature_barplot(appointments_features, "appointments_last_days")
plt.show()

## Calculate patient features

### Postal codes

Using the dump from https://download.geonames.org/export/zip/

In [ ]:
all_postalcodes = process_postal_codes("../data/raw/NL.txt")
appointments_features = add_patient_features(appointments_features, all_postalcodes)

### Plot patient features

In [ ]:
feature_barplot(appointments_features, "age")
plt.show()

In [ ]:
feature_scatter(
    appointments_features,
    "dist_umcu",
    feature_name="distance to UMCU",
    round_feature=True,
)
plt.show()

## Calculate punctuality patients

In [ ]:
appointments_features = add_minutes_early(appointments_features)

In [ ]:
appointments_features["minutes_early"].plot.hist(bins=10)

### Plot punctuality vs no show

In [ ]:
appointments_features["prev_minutes_early_na"] = appointments_features[
    "prev_minutes_early"
].isna()
appointments_features.groupby("no_show")["prev_minutes_early_na"].value_counts(
    normalize=True
).unstack()

In [ ]:
feature_scatter(
    appointments_features,
    "prev_minutes_early",
    feature_name="previous minutes too early",
    round_feature=True,
)
plt.show()

## Add time features

In [ ]:
appointments_features = add_time_features(appointments_features)

In [ ]:
feature_barplot(appointments_features, "hour")
plt.show()

In [ ]:
feature_barplot(appointments_features, "weekday")
plt.show()

## Plot categorical features

In [ ]:
ax = feature_barplot(appointments_features, "specialty_code")
plt.show()